In [46]:
a = 'hongseungki'
b = 'abcdefghijk'
print a[:3]
print b[:3]
print a[:3] + b[:3]

hon
abc
honabc


In [61]:
import csv
pairwise = dict()
paths = ['train.tsv', 'devel.tsv']
sentences = []
sentence = []
p_label = ''
p_token = ''
for path in paths:
    with open(path) as fin:
        reader = csv.reader(fin, delimiter='\t', quoting=csv.QUOTE_NONE)
        for i, row in enumerate(reader):
            if i == 0:
                pass
            if len(row) < 2:
                sentences.append(sentence)
                sentence = []
            else:
                token = row[0]
                label = row[1]
                if len(p_label) > 0:
                    key_v = p_label[:3] + label[:3]
                    if key_v in pairwise:
                        pairwise[key_v] += 1
                    else:
                        pairwise[key_v] = 1
                p_label = label

for key, value in pairwise.items():
    if value > 10:
        continue
    if key[0] is 'O':
        print key[0], '-', key[1:], ' = ', value
    else:
        print key[:3], '-', key[3:], ' = ', value
        
print len(pairwise.keys())

E-C - S-C  =  10
E-D - S-C  =  2
E-D - S-D  =  2
E-C - B-C  =  1
E-C - B-D  =  1
E-D - B-D  =  3
S-D - S-C  =  4
S-C - B-D  =  8
S-C - B-C  =  2
S-D - B-D  =  9
31


In [19]:
import csv
pairwise = dict()
# paths = ['train.tsv', 'devel.tsv']
paths = ['train.tsv', 'devel.tsv', 'test.tsv']
sentences = []
sentence = []
p_label = ''
p_token = ''

all_length = []
length = 0
longer_entity = 0
max_length_disease = 0
max_length_disease = 0
entity_count = 0

for path in paths:
    with open(path) as fin:
        reader = csv.reader(fin, delimiter='\t', quoting=csv.QUOTE_NONE)
        entity_length_turn_on = False
        for i, row in enumerate(reader):
            if i == 0:
                pass
            if len(row) < 2:
                sentences.append(sentence)
                sentence = []
            else:
                token = row[0]
                label = row[1]
                if len(p_label) == 1 and len(label) > 1:
                    entity_length_turn_on = True
                    length = 1
                elif entity_length_turn_on == True and len(label) > 1:
                    length += 1
                elif entity_length_turn_on == True and len(label) == 1:
                    entity_length_turn_on = False
                    all_length.append(length)
                    if length > 1:
                        longer_entity += 1
                    if length > max_length:
                        max_length = length
                        max_entity_token = p_token
                    entity_count += 1
                    length = 0
                p_label = label
                p_token = token

total = sum(all_length, 0.0)
average_length = total / entity_count
print entity_count
print longer_entity
print total
print max_length, max_entity_token
print average_length
print len(sentences)
                

28416
7693
42440.0
56 carboxylate
1.49352477477
13935


In [117]:
import csv
import operator
pairwise2 = dict()
sentence_scored = dict()
paths = ['test.tsv']
paths = ['train.tsv', 'devel.tsv']
sentences = []
sentence = []
label_sentence = []

def scoring(labels):
    eps = 1e-5
    score = 0.
    n_O = 0
    n_non_O = 0
    for label in labels:
        if label.startswith('B-C') or label.startswith('E-C'):
            score += 0.25
        if label.startswith('B-D') or label.startswith('I-D') or label.startswith('E-D'):
            score += 0.15
        if label.startswith('I-C'):
            score += 0.1
        if label.startswith('S'):
            score -= 5.
        if label.startswith('O'):
            n_O += 1
        else:
            n_non_O += 1
    score += n_non_O / (n_O + eps)
    score = score if len(labels) >= 5 else 0
    return score

sentence_count = 0
for path in paths:
    with open(path) as fin:
        reader = csv.reader(fin, delimiter='\t', quoting=csv.QUOTE_NONE)
        for i, row in enumerate(reader):
            if i == 0:
                print i, row
                pass
            if len(row) < 2:
                score = scoring(label_sentence)
                sentence_label = (sentence, label_sentence, score)
                sentences.append(sentence_label)
                sentence_scored[sentence_count] = sentence_label
                sentence = []
                label_sentence = []
                sentence_count += 1
            else:
                token = row[0]
                label = row[1]
                sentence.append(token)
                label_sentence.append(label)

print len(sentences)
n_sampled = 0
for idx, sentence_label in sentence_scored.items():
    if sentence_label[2] >= 0.8:
        n_sampled += 1
        sentences.append(sentence_label)
        sentences.append(sentence_label)

print '# of sampled = ', n_sampled
print len(sentences)

p_label = ''
for sentence_label in sentences:
    labels = sentence_label[1]
    for label in labels:
        if len(p_label) > 0:
            key_v = p_label[:3] + label[:3]
            if key_v in pairwise2:
                pairwise2[key_v] += 1
            else:
                pairwise2[key_v] = 1
        p_label = label

p_label = ''
for sentence_label in sentences:
    labels = sentence_label[1]
    for label in labels:
        if len(p_label) > 0:
            key_v = p_label[:3] + label[:3]
            if key_v in pairwise2:
                pairwise2[key_v] += 1
            else:
                pairwise2[key_v] = 1
        p_label = label

count = 0

for key, value in pairwise2.items():
    if value > 20:
        continue
    count += 1
    if key[0] is 'O':
        print key[0], '-', key[1:], ' = ', value
    else:
        print key[:3], '-', key[3:], ' = ', value

print count
count = 0
for key, value in pairwise2.items():
    if value <= 20:
        continue
    count += 1
    if key[0] is 'O':
        print key[0], '-', key[1:], ' = ', value
    else:
        print key[:3], '-', key[3:], ' = ', value
print count

print len(pairwise2.keys())

pairwise_sorted = sorted(pairwise2.items(), key=operator.itemgetter(1))

print 'Sorted'
for key, value in pairwise_sorted[::-1]:
    if key[0] is 'O':
        print key[0], '-', key[1:], ' = ', value
    else:
        print key[:3], '-', key[3:], ' = ', value



0 ['Selegiline', 'S-Chemical']
0 ['22', 'B-Chemical']
9139
# of sampled =  347
9833
E-C - S-D  =  15
E-C - S-C  =  10
E-D - S-C  =  2
E-D - S-D  =  2
S-C - S-C  =  16
E-C - B-C  =  1
E-C - B-D  =  1
E-D - B-D  =  3
S-D - S-C  =  4
S-D - S-D  =  15
S-C - B-D  =  8
S-C - B-C  =  2
S-D - B-D  =  9
13
O - S-C  =  9016
B-C - I-C  =  1058
I-D - I-D  =  1443
I-C - E-C  =  1058
O - S-D  =  4745
O - B-D  =  4337
S-D - O  =  4893
O - B-C  =  1985
E-D - O  =  4351
S-C - S-D  =  144
I-D - E-D  =  1544
B-D - E-D  =  2814
O - O  =  199050
E-C - O  =  1961
B-C - E-C  =  930
S-C - O  =  8879
I-C - I-C  =  1876
B-D - I-D  =  1544
18
31
Sorted
O - O  =  199050
O - S-C  =  9016
S-C - O  =  8879
S-D - O  =  4893
O - S-D  =  4745
E-D - O  =  4351
O - B-D  =  4337
B-D - E-D  =  2814
O - B-C  =  1985
E-C - O  =  1961
I-C - I-C  =  1876
B-D - I-D  =  1544
I-D - E-D  =  1544
I-D - I-D  =  1443
I-C - E-C  =  1058
B-C - I-C  =  1058
B-C - E-C  =  930
S-C - S-D  =  144
S-C - S-C  =  16
S-D - S-D  =  15
E-C - S-D 

In [44]:
import csv
pairwise = dict()
path = 'devel.tsv'
sentences = []
sentence = []
previous_label = ''
previous_token = ''
count_b_i = 0
count_i_i = 0
count_i_e = 0
count_b_e = 0
count_s_b = 0
count_e_s = 0
count_s_s = 0
count_e_b = 0
with open(path) as fin:
    reader = csv.reader(fin, delimiter='\t', quoting=csv.QUOTE_NONE)
    for i, row in enumerate(reader):
        if i == 0:
#             print row[0], row[1]
            pass
        if len(row) < 2:
            sentences.append(sentence)
            sentence = []
        else:
            token = row[0]
            label = row[1]
            if previous_label.startswith('B-') and label.startswith('I-'):
                count_b_i += 1
            if previous_label.startswith('I-') and label.startswith('I-'):
                count_i_i += 1
            if previous_label.startswith('I-') and label.startswith('E-'):
                count_i_e += 1
            if previous_label.startswith('B-') and label.startswith('E-'):
                count_b_e += 1
            if previous_label.startswith('S-') and label.startswith('B-'):
                count_s_b += 1
                print previous_token, token
            if previous_label.startswith('E-') and label.startswith('S-'):
                count_e_s += 1
            if previous_label.startswith('S-') and label.startswith('S-'):
                count_s_s += 1
            if previous_label.startswith('E-') and label.startswith('B-'):
                count_e_b += 1            
            previous_token = row[0]
            previous_label = row[1]
            sentence.append(token)
            sentence.append(label)
#             print token, label
    print i
    print 'B-I', count_b_i
    print 'I-I', count_i_i
    print 'I-E', count_i_e
    print 'B-E', count_b_e
    print 'S-B', count_s_b
    print 'E-S', count_e_s
    print 'S-S', count_s_s
    print 'E-B', count_e_b

ischemic cardiac
asystolic cardiac
parkinsonian muscle
painful erythematous
Amiodarone pulmonary
ischemic chest
BHA forestomach
hypertensive coronary
CM myocardial
hyperkinetic abnormal
122032
B-I 956
I-I 1057
I-E 956
B-E 1502
S-B 10
E-S 9
S-S 107
E-B 4
